In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import shapely

In [2]:
cellrebel = pd.read_csv('Source Data/Bogor_Bekasi_RSRP_Throughput.csv', low_memory=False)
# sites = pd.read_csv('2023_Bekasi.csv')
grid = gpd.read_file('Grid Folder/grid_bogorbekasi_250x250.geojson')
boundary = gpd.read_file('Polygon Kecamatan_Yunan/DESA_83218_EID_kec_comb1.geojson')

In [3]:
cellrebel.drop(['dl_throughput', 'ul_throughput', 'latency'], axis=1, inplace=True, errors='ignore')
cellrebel = cellrebel[cellrebel['mobile_operator'].str.contains('Indosat Ooredoo|XL Axiata')==True]
cellrebel = cellrebel[~cellrebel['reference_signal_received_power'].isnull()]

In [4]:
cellrebel_gpd = gpd.GeoDataFrame(cellrebel, geometry=gpd.points_from_xy(cellrebel.longitude, cellrebel.latitude), crs=4326)
# sites_gpd = gpd.GeoDataFrame(sites, geometry=gpd.points_from_xy(sites['Long'], sites['Lat']), crs=4326)
grid = grid.to_crs(4326)

In [5]:
### Create Rectangle Grid 100x100 metre from mininum/maximum long-lat in CRS:3426
### These projection are imagined earth as flat object. these are not accurate for higher degree of long-lat
### Based on plotting in QGIS, the difference of grid reach 50 meter in latitude and 25 meter in longitude
### Try to convert them to EPSG:3857 

# xmin, ymin, xmax, ymax = cellrebel_gpd.total_bounds

# length =  0.0008985 ### (the distance for 100 metres, so if you want to get 250 metres, need to multiply 2.5)
# wide =  0.0008985 ### (the distance for 100 metres, so if you want to get 250 metres, need to multiply 2.5)

# cols = list(np.arange(xmin, xmax + wide, wide))
# rows = list(np.arange(ymin, ymax + length, length))

# polygons = []
# for x in cols[:-1]: ### [:-1] these are used stop -1 of position from the end of list 
#     for y in rows[:-1]:
#         polygons.append(Polygon([(x,y), (x+wide, y), (x+wide, y+length), (x, y+length)]))

# grid = gpd.GeoDataFrame({'geometry':polygons}, crs=4326)

# grid.reset_index(inplace=True)
# grid.rename(columns={'index':'id'}, inplace = True)


In [6]:
grid['geometry1'] = grid['geometry']

In [7]:
points_within = gpd.sjoin(left_df= cellrebel_gpd,right_df=grid, how='left', predicate='within')
points_within.drop(['index_right'], axis=1, inplace=True)
points_within = gpd.sjoin(left_df= points_within, right_df=boundary, how='left', predicate='within')

In [8]:
pivot = pd.pivot_table(points_within, index= ['id', 'KAB_KOT', 'KEC'],
                        columns=['mobile_operator'], values=['reference_signal_received_power'], aggfunc=np.mean)

In [9]:
pivot

reference_signal_received_power          
mobile_operator                            Indosat Ooredoo XL Axiata
id       KAB_KOT KEC                                                
606.0    BOGOR   JASINGA                        -93.733333       NaN
1353.0   BOGOR   JASINGA                       -107.000000       NaN
1354.0   BOGOR   JASINGA                       -106.000000       NaN
1379.0   BOGOR   JASINGA                       -115.500000       NaN
1383.0   BOGOR   JASINGA                       -101.333333       NaN
...                                                    ...       ...
147828.0 BEKASI  PEBAYURAN                             NaN    -108.0
147829.0 BEKASI  PEBAYURAN                      -92.000000       NaN
147830.0 BEKASI  PEBAYURAN                      -94.727273       NaN
147831.0 BEKASI  PEBAYURAN                      -98.333333       NaN
147832.0 BEKASI  PEBAYURAN                      -94.071429       NaN

[23747 rows x 2 columns]

In [10]:
pivot.reset_index(inplace=True, col_level=1, allow_duplicates=False)
pivot = pivot.droplevel(level=0, axis=1)

In [11]:
# Methods-1 : replacing np.nan in the sample to -120 to increase number of grid covered

# pivot['Indosat Ooredoo'] = pivot['Indosat Ooredoo'].replace(np.nan, -120)
# pivot['XL Axiata'] = pivot['XL Axiata'].replace(np.nan, -120)

In [12]:
# Methods-2 : drop nan in the sample. The impact : qty of compared grid will decrease

pivot = pivot[~pivot['Indosat Ooredoo'].isnull() & ~pivot['XL Axiata'].isnull()]
pivot = pivot[(pivot['Indosat Ooredoo']) < 0 & (pivot['XL Axiata'] < 0)]

In [13]:
pivot

mobile_operator,id,KAB_KOT,KEC,Indosat Ooredoo,XL Axiata
15,2472.0,BOGOR,JASINGA,-110.121622,-107.000000
17,2845.0,BOGOR,JASINGA,-107.000000,-107.000000
44,5406.0,BOGOR,TENJO,-103.866667,-114.000000
62,5779.0,BOGOR,TENJO,-118.812500,-115.846154
63,5780.0,BOGOR,TENJO,-113.000000,-117.625000
...,...,...,...,...,...
23719,146701.0,BEKASI,PEBAYURAN,-83.857143,-95.714286
23720,146703.0,BEKASI,PEBAYURAN,-103.000000,-103.384615
23725,147059.0,BEKASI,PEBAYURAN,-97.636364,-101.000000
23730,147087.0,BEKASI,PEBAYURAN,-80.500000,-125.000000


In [14]:
pivot['1-1-IOH-XL'] = 'Yes'
points_within = points_within.merge(pivot[['id', '1-1-IOH-XL']], how='left', on='id', suffixes=['_x'])
points_within = points_within[points_within['1-1-IOH-XL'] == 'Yes']

In [ ]:
# pivot.loc[pivot['XL Axiata'] > pivot['Indosat Ooredoo'], 'compare'] = 'XL Wins'
# pivot.loc[pivot['XL Axiata'] == pivot['Indosat Ooredoo'], 'compare'] = 'Same'
# pivot.loc[pivot['XL Axiata'] < pivot['Indosat Ooredoo'], 'compare'] = 'XL Lose'

In [ ]:
pivot['compare'] = pivot['XL Axiata'] - pivot['Indosat Ooredoo']

In [ ]:
pivot = pivot.merge(grid[['id', 'geometry1']], how='left', on='id')

In [ ]:
pivot_gpd = gpd.GeoDataFrame(pivot, geometry=pivot['geometry1'], crs=4326)

In [ ]:
pivot_gpd.drop(['geometry1'], axis=1, inplace=True)

In [ ]:
# grid.to_csv('hasil.csv')
pivot.to_csv('result/bogorbekasi_rsrp_250x250_methods2.csv')
pivot_gpd.to_file('result/bogorbekasi_rsrp_250x250_methods2.json', driver="GeoJSON", na='drop')


In [ ]:
points_within.to_csv('bogorbekasi_rsrp_processed_methods2.csv')

In [ ]:
### Methods to count sample for each grid

pivot1 = pd.pivot_table(points_within, index= ['id', 'KAB_KOT', 'KEC'],
                        columns=['mobile_operator'], values=['reference_signal_received_power'], aggfunc = len)

pivot1.reset_index(inplace=True, col_level=1, allow_duplicates=False)
pivot1 = pivot1.droplevel(level=0, axis=1)

pivot1 = pivot1[~pivot1['Indosat Ooredoo'].isnull() & ~pivot1['XL Axiata'].isnull()]
pivot1 = pivot1[(pivot1['Indosat Ooredoo'] > 0) & (pivot1['XL Axiata'] > 0)]

pivot1.loc["Indosat Ooredoo"] = pivot1.sum()